In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd

data_dir = "Bean_Dataset/Bean_Dataset"
img_height, img_width = 128, 128
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    data_dir, target_size=(img_height, img_width),
    batch_size=batch_size, class_mode='categorical', subset='training'
)

val_gen = datagen.flow_from_directory(
    data_dir, target_size=(img_height, img_width),
    batch_size=batch_size, class_mode='categorical', subset='validation'
)

num_classes = len(train_gen.class_indices)

dnn_model = Sequential([
    Flatten(input_shape=(img_height, img_width, 3)),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])
dnn_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])
cnn_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

dnn_history = dnn_model.fit(train_gen, epochs=20, validation_data=val_gen)
cnn_history = cnn_model.fit(train_gen, epochs=20, validation_data=val_gen)

comparison = pd.DataFrame({
    'Model': ['DNN', 'CNN'],
    'Parameters': [dnn_model.count_params(), cnn_model.count_params()],
    'Final Train Accuracy': [
        dnn_history.history['accuracy'][-1],
        cnn_history.history['accuracy'][-1]
    ],
    'Final Val Accuracy': [
        dnn_history.history['val_accuracy'][-1],
        cnn_history.history['val_accuracy'][-1]
    ]
})

print(comparison)

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(dnn_history.history['accuracy'], label='DNN Train')
plt.plot(dnn_history.history['val_accuracy'], label='DNN Val')
plt.plot(cnn_history.history['accuracy'], label='CNN Train')
plt.plot(cnn_history.history['val_accuracy'], label='CNN Val')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(dnn_history.history['loss'], label='DNN Train')
plt.plot(dnn_history.history['val_loss'], label='DNN Val')
plt.plot(cnn_history.history['loss'], label='CNN Train')
plt.plot(cnn_history.history['val_loss'], label='CNN Val')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()